In [1]:
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [2]:
X,y = make_classification(n_samples=10000, n_features=10, n_informative=3)

In [3]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [4]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)

print('Decision Tree accuracy',accuracy_score(y_test,y_pred))

Decision Tree accuracy 0.911


## Bagging

In [5]:
bag = BaggingClassifier(
    base_estimator = DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.5,
    bootstrap=True,
    random_state=42
)

In [6]:
bag.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples=0.5,
                  n_estimators=500, random_state=42)

In [7]:
y_pred = bag.predict(X_test)

In [8]:
accuracy_score(y_test,y_pred)

0.9335

In [10]:
bag.estimators_samples_[0].shape

(4000,)

In [11]:
bag.estimators_features_[0].shape

(10,)

## Bagging using SVM

In [12]:
bag = BaggingClassifier(
    base_estimator=SVC(),
    n_estimators=500,
    max_samples=.25,
    bootstrap=True,
    random_state=42
)

In [14]:
bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print('Bagging using SVM',accuracy_score(y_test,y_pred))

Bagging using SVM 0.916


## Pasting

In [15]:
bag = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=False,
    random_state=42,
    verbose=1,
    n_jobs=-1
)

In [17]:
bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print('Pasting Classifer',accuracy_score(y_test,y_pred))

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   2 out of  12 | elapsed:    3.0s remaining:   15.3s
[Parallel(n_jobs=12)]: Done  12 out of  12 | elapsed:    3.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   2 out of  12 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=12)]: Done  12 out of  12 | elapsed:    0.0s finished


Pasting Classifer 0.9325


## Random Subspaces

In [18]:
bag = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=1.0,
    bootstrap=False,
    max_features=0.5,
    bootstrap_features=True,
    random_state=42
)

In [20]:
bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Random Subspaces",accuracy_score(y_test,y_pred))

Random Subspaces 0.93


## Random Patches

In [21]:
bag = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    max_features=0.5,
    bootstrap_features=True,
    random_state=42
)

In [22]:
bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Random Patches Classifer",accuracy_score(y_test,y_pred))

Random Patches Classifer 0.933


## OOB

In [23]:
bag = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    oob_score=True,
    random_state=42
)

In [24]:
bag.fit(X_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples=0.25,
                  n_estimators=500, oob_score=True, random_state=42)

In [25]:
bag.oob_score_

0.941125

In [26]:
y_pred = bag.predict(X_test)
print("Accuracy",accuracy_score(y_test,y_pred))

Accuracy 0.9315


## Bagging Tips
- Bagging generally gives better results than Pasting
- Good results come around the 25% to 50% row sampling mark
- Random patches and subspaces should be used while dealing with high dimensional data
- To find the correct hyperparameter values we can do GridSearchCV/RandomSearchCV

## Applying GridSearchCV

In [27]:
from sklearn.model_selection import GridSearchCV

In [28]:
parameters = {
    'n_estimators':[50,100,500],
    'max_samples':[0.1,0.4,0.7,1.0],
    'max_features':[0.1,0.4,0.7,1.0]
}

In [29]:
search = GridSearchCV(BaggingClassifier(),parameters,cv=5)

In [30]:
search.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=BaggingClassifier(),
             param_grid={'max_features': [0.1, 0.4, 0.7, 1.0],
                         'max_samples': [0.1, 0.4, 0.7, 1.0],
                         'n_estimators': [50, 100, 500]})

In [31]:
search.best_params_

{'max_features': 0.7, 'max_samples': 0.7, 'n_estimators': 100}

In [32]:
search.best_score_

0.9428749999999999